In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import seaborn as sns

In [8]:
df = pd.read_csv('../not_on_git/mw_cat/chandra_filtered_sources.csv' , index_col = 'name')
df_id = pd.read_csv('compiled_data_v3/id_frame.csv' , index_col='name')[['offset' , 'class']]
off = 1.01 
df = pd.merge(df_id[df_id['offset']<off] , df , left_index=True , right_index =True , how='right')
df = df.drop(columns = ['significance'  , 'offset' , 'ra' , 'dec' , 'var_flag' , 'var_inter_hard_flag' , 'likelihood'])
df = df.rename(columns = {
    'flux_aper_b' : 'b-csc' , 
    'flux_aper_h' : 'h-csc' ,
    'flux_aper_m': 'm-csc' ,
    'flux_aper_s': 's-csc' ,
    'flux_aper_u': 'u-csc' ,
})
df['class'] = df['class'].replace(np.nan , 'X')
df = pd.merge(
    df , pd.read_csv('mw_cat/sdss.csv' , index_col='name') ,
    left_index=True , 
    right_index = True , 
    how = 'left'
)
df = pd.merge(
    df , pd.read_csv('mw_cat/wise_combined.csv' , index_col='name') ,
    left_index=True , 
    right_index = True , 
    how = 'left'
)
df = pd.merge(
    df , pd.read_csv('mw_cat/galex_combined.csv' , index_col='name') ,
    left_index=True , 
    right_index = True , 
    how = 'left'
)
df = pd.merge(
    df , pd.read_csv('mw_cat/gaia.csv' , index_col='name') ,
    left_index=True , 
    right_index = True , 
    how = 'left'
)
df

,class,gal_l,gal_b,b-csc,h-csc,m-csc,s-csc,u-csc,hard_hm,hard_hs,...,zmag,W1mag,W2mag,W3mag,W4mag,fuv_mag,nuv_mag,g_mag,bp_mag,rp_mag
name,,,,,,,,,,,,,,,,,,,,,
2CXO J003935.9-732725,X,304.096050,-43.644346,0.000000e+00,7.141104e-15,1.529669e-15,0.000000e+00,0.000000e+00,0.231106,-0.058713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J003936.7-731249,X,304.116127,-43.887309,1.458552e-14,1.196830e-14,2.057132e-15,0.000000e+00,NaN,0.343535,-0.016240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004028.7-731106,X,304.031850,-43.919668,1.094959e-13,8.183634e-14,1.744202e-14,1.216939e-14,NaN,0.269831,0.336040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004506.3-730056,X,303.575841,-44.104667,2.379848e-14,1.178519e-14,7.671868e-15,4.860946e-15,6.986833e-14,-0.106184,-0.066209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004659.0-731918,X,303.374710,-43.802532,0.000000e+00,0.000000e+00,1.259314e-15,6.172243e-16,8.098542e-14,0.151156,0.394753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2CXO J220613.7-495727,X,345.582863,-51.190099,3.452933e-14,2.859357e-14,7.190546e-15,0.000000e+00,NaN,-0.013741,0.224859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220614.6-500951,X,345.272648,-51.119708,7.855447e-15,2.507461e-15,0.000000e+00,5.262109e-15,0.000000e+00,0.999375,-0.570893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220618.4-500554,X,345.365084,-51.152644,2.417635e-14,1.919246e-14,2.265201e-15,3.881337e-15,0.000000e+00,0.463460,0.056215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
